In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score


df = pd.read_csv("training.csv")
df.head(3)

,class,GLCM_pan,Mean_Green,Mean_Red,Mean_NIR,SD_pan
0,w,120.362774,205.500000,119.395349,416.581395,20.676318
1,w,124.739583,202.800000,115.333333,354.333333,16.707151
2,w,134.691964,199.285714,116.857143,477.857143,22.496712


In [11]:
df.shape

(4339, 6)

In [12]:
df.iloc[:, 0].value_counts()

n    4265
w      74
Name: class, dtype: int64

In [13]:
df.replace({'class': {'w': 1, 'n': 0}}, inplace=True)


In [14]:
df.iloc[:, 0].value_counts()


0    4265
1      74
Name: class, dtype: int64

In [15]:
df.describe()


,class,GLCM_pan,Mean_Green,Mean_Red,Mean_NIR,SD_pan
count,4339.000000,4339.000000,4339.000000,4339.000000,4339.000000,4339.000000
mean,0.017055,126.831298,233.906908,117.292439,534.104683,24.924588
std,0.129490,13.735836,60.757687,60.711159,154.495500,11.008303
min,0.000000,0.000000,164.625000,59.142857,86.500000,0.000000
25%,0.000000,118.589080,206.000000,91.975244,422.875000,18.009143
50%,0.000000,127.479167,221.454545,101.727273,528.500000,23.612444
75%,0.000000,135.043591,241.791304,116.866071,643.087037,29.899148
max,1.000000,183.281250,955.714286,746.333333,1005.516129,156.508431


In [16]:
cols = list(set(df.iloc[:, 1:].columns))
base_cols = cols

In [17]:
def transform_data(data):
    for i1, col1 in enumerate(base_cols):
        data[col1 + '_log'] = np.log(data[col1] + 1.1)

        for i2, col2 in enumerate(base_cols):
            data['%s_%s_1' % (col1, col2)] = data[col1] - data[col2]
            data['%s_%s_2' % (col1, col2)] = data[col1] + data[col2]
            data['%s_%s_3' % (col1, col2)] = data[col1] / (data[col2] + 0.1)
            data['%s_%s_4' % (col1, col2)] = data[col1] * data[col2]

            data['%s_%s_11' % (col1, col2)] = data[col1] - np.log(data[col2] + 1)
            data['%s_%s_22' % (col1, col2)] = data[col1] + np.log(data[col2] + 1)
            data['%s_%s_33' % (col1, col2)] = data[col1] / (np.log(data[col2] + 1) + 0.1)
            data['%s_%s_44' % (col1, col2)] = data[col1] * np.log(data[col2] + 1)

    return data


df_tr = transform_data(df)

/var/folders/g_/9v3q8zfn4dj84j48wgjpsd7c0000gn/T/ipykernel_5153/1557307549.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['%s_%s_44' % (col1, col2)] = data[col1] * np.log(data[col2] + 1)
/var/folders/g_/9v3q8zfn4dj84j48wgjpsd7c0000gn/T/ipykernel_5153/1557307549.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['%s_%s_1' % (col1, col2)] = data[col1] - data[col2]
/var/folders/g_/9v3q8zfn4dj84j48wgjpsd7c0000gn/T/ipykernel_5153/1557307549.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

In [18]:
df_tr.head(3)


,class,GLCM_pan,Mean_Green,Mean_Red,Mean_NIR,SD_pan,Mean_NIR_log,Mean_NIR_Mean_NIR_1,Mean_NIR_Mean_NIR_2,Mean_NIR_Mean_NIR_3,...,Mean_Red_SD_pan_33,Mean_Red_SD_pan_44,Mean_Red_Mean_Red_1,Mean_Red_Mean_Red_2,Mean_Red_Mean_Red_3,Mean_Red_Mean_Red_4,Mean_Red_Mean_Red_11,Mean_Red_Mean_Red_22,Mean_Red_Mean_Red_33,Mean_Red_Mean_Red_44
0,1,120.362774,205.500000,119.395349,416.581395,20.676318,6.034719,0.0,833.162791,0.999760,...,37.590386,367.286401,0.0,238.790698,0.999163,14255.249315,114.604568,124.186130,24.412328,571.996957
1,1,124.739583,202.800000,115.333333,354.333333,16.707151,5.873338,0.0,708.666667,0.999718,...,38.780952,331.464374,0.0,230.666667,0.999134,13301.777770,110.576874,120.089793,23.748439,548.578344
2,1,134.691964,199.285714,116.857143,477.857143,22.496712,6.171611,0.0,955.714286,0.999791,...,35.880304,368.901697,0.0,233.714286,0.999145,13655.591847,112.087670,121.626616,23.997902,557.347016


In [20]:
x_data = df.iloc[:,1:]
y_data = df.iloc[:,0]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)


In [22]:
model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [23]:
metrics = []
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    metrics.append([f1, roc, rec, prc])
    
evaluate_results(y_test, y_predict)

Classification results:
f1: 90.00%
roc: 94.94%
recall: 90.00%
precision: 90.00%


In [24]:
mod_data = df_tr.copy()
#mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,0].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 19/74 as positives and unlabeling the rest


In [25]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    4320
 1      19
Name: class_test, dtype: int64


In [26]:
mod_data.head(10)


,class,GLCM_pan,Mean_Green,Mean_Red,Mean_NIR,SD_pan,Mean_NIR_log,Mean_NIR_Mean_NIR_1,Mean_NIR_Mean_NIR_2,Mean_NIR_Mean_NIR_3,...,Mean_Red_SD_pan_44,Mean_Red_Mean_Red_1,Mean_Red_Mean_Red_2,Mean_Red_Mean_Red_3,Mean_Red_Mean_Red_4,Mean_Red_Mean_Red_11,Mean_Red_Mean_Red_22,Mean_Red_Mean_Red_33,Mean_Red_Mean_Red_44,class_test
0,1,120.362774,205.500000,119.395349,416.581395,20.676318,6.034719,0.0,833.162791,0.999760,...,367.286401,0.0,238.790698,0.999163,14255.249315,114.604568,124.186130,24.412328,571.996957,1
1,1,124.739583,202.800000,115.333333,354.333333,16.707151,5.873338,0.0,708.666667,0.999718,...,331.464374,0.0,230.666667,0.999134,13301.777770,110.576874,120.089793,23.748439,548.578344,-1
2,1,134.691964,199.285714,116.857143,477.857143,22.496712,6.171611,0.0,955.714286,0.999791,...,368.901697,0.0,233.714286,0.999145,13655.591847,112.087670,121.626616,23.997902,557.347016,-1
3,1,127.946309,178.368421,92.368421,278.473684,14.977453,5.633266,0.0,556.947368,0.999641,...,255.969387,0.0,184.736842,0.998919,8531.925207,87.831868,96.904974,19.921786,419.034255,-1
4,1,135.431548,197.000000,112.690476,532.952381,17.604193,6.280494,0.0,1065.904762,0.999812,...,329.437869,0.0,225.380952,0.999113,12699.143426,107.956997,117.423956,23.314565,533.418074,-1
5,1,118.347962,226.150000,138.850000,608.900000,29.072797,6.413459,0.0,1217.800000,0.999836,...,472.592775,0.0,277.700000,0.999280,19279.322500,133.909430,143.790570,27.546486,685.998203,-1
6,1,135.436282,184.500000,95.142857,309.190476,13.055264,5.737509,0.0,618.380952,0.999677,...,251.462283,0.0,190.285714,0.998950,9052.163265,90.577022,99.708692,20.391388,434.406604,1
7,1,121.169643,226.000000,146.214286,595.571429,22.808542,6.391367,0.0,1191.142857,0.999832,...,463.505782,0.0,292.428571,0.999317,21378.617343,141.222396,151.206175,28.715135,729.885521,-1
8,1,131.127161,232.784314,144.588235,563.843137,11.948563,6.336725,0.0,1127.686275,0.999823,...,370.288279,0.0,289.176471,0.999309,20905.757787,139.607453,149.569018,28.457868,720.162528,1
9,1,134.498092,210.212121,116.909091,594.848485,27.937685,6.390154,0.0,1189.696970,0.999832,...,393.416010,0.0,233.818182,0.999145,13667.735535,112.139177,121.679005,24.006398,557.646299,-1


In [27]:
x_data = mod_data.iloc[:,1:-1].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,0].values # original class

1. RNS

In [28]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(19, 212) (19, 212)


In [29]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,1:-1].values, 
          sample_train.iloc[:,0].values)
y_predict = model.predict(sample_test.iloc[:,1:-1].values)
evaluate_results(sample_test.iloc[:,0].values, y_predict)

Classification results:
f1: 38.93%
roc: 95.37%
recall: 94.44%
precision: 24.52%


In [31]:
metrics_df = pd.DataFrame(metrics,
                          columns = ['f-score', 'roc_auc', 'recall', 'precision'],
                          index = ['normal_model', 'PU_learning_model'])
metrics_df

,f-score,roc_auc,recall,precision
normal_model,0.900000,0.949417,0.900000,0.900000
PU_learning_model,0.389313,0.953739,0.944444,0.245192


In [32]:
mod_data = df_tr.copy()
pos_ind = np.where(mod_data.iloc[:,0].values == 1)[0]
np.random.shuffle(pos_ind)
metrics1=[]
alpha_list = np.linspace(0.1, 0.9, 9)
for alpha in alpha_list:
    pos_sample_len = int(np.ceil(alpha * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    
    model = xgb.XGBClassifier()

    model.fit(sample_train.iloc[:,1:-1].values, sample_train.iloc[:,0].values)
    y_predict = model.predict(sample_test.iloc[:,1:-1].values)
    y_test1= sample_test.iloc[:,0].values
    f1 = f1_score(y_test1, y_predict)
    roc = roc_auc_score(y_test1, y_predict) 
    rec = recall_score(y_test1, y_predict, average='binary') 
    prc = precision_score(y_test1, y_predict, average='binary')
    
    metrics1.append([alpha,f1, roc, rec, prc])

In [33]:
metrics_df1 = pd.DataFrame(metrics1,
                          columns = ['alpha','f-score', 'roc_auc', 'recall', 'precision'])
metrics_df1

,alpha,f-score,roc_auc,recall,precision
0,0.1,0.340206,0.969932,1.0,0.204969
1,0.2,0.339181,0.973418,1.0,0.204225
2,0.3,0.267380,0.967712,1.0,0.154321
3,0.4,0.226221,0.964463,1.0,0.127536
4,0.5,0.284615,0.978004,1.0,0.165919
5,0.6,0.192691,0.971209,1.0,0.106618
6,0.7,0.265060,0.985521,1.0,0.152778
7,0.8,0.118644,0.975268,1.0,0.063063
8,0.9,0.110236,0.986541,1.0,0.058333
